# Graph DB Neo4j chain
지식 그래프 데이터베이스로 많이 사용되는 Neo4j와 LLM을 함께 사용하는 방법에 대해 알아봅니다

## Python Package
예시에서 사용할 몇 가지 주요 패키지를 설치합니다. 아래 주석처리된 명령어를 통해 Jupyter notebook 환경에서 바로 설치하실 수 있습니다.

In [15]:
# !pip install langchain -U
# !pip install neo4j
# !pip install python-dotenv

## Neo4j 설치
Docker 환경에서 Neo4j 데이터베이스 서버를 활성화 합니다. 
아래 Docker 컨테이너 실행 명령어를 통해 Neo4j를 설치하고 서버를 활성화 합니다. 
```
docker run \
--name neo4j \
-p 7474:7474 -p 7687:7687 \
-d \
-e NEO4J_AUTH=neo4j/pleaseletmein \
-e NEO4J_PLUGINS=\[\"apoc\"\]  \
-e NEO4J_apoc_export_file_enabled=true \
-e NEO4J_apoc_import_file_enabled=true \
-e NEO4J_apoc_import_file_use__neo4j__config=true \
neo4j:latest
```
위 명령어를 실행하면 Neo4j 서버가 실행되고, http://localhost:7474/ 에서 Neo4j Browser를 통해 Neo4j 서버에 접속할 수 있습니다.
참고로 Neo4j Browser의 기본 계정은 neo4j/neo4j 이지만 옵션으로 설정한 NEO4J_AUTH=neo4j/pleaseletmein 을 통해 neo4j/pleaseletmein 계정으로 접속할 수 있습니다.

## LangChain with Neo4j
- LangChain 패키지에서 지원하는 Neo4j 연결 작업을 수행합니다
- 예시 쿼리를 통해 신규 그래프 데이터를 추가하고 Neo4j 서버에 저장합니다
- LangChain으로 연결된 Neo4j 전용 Agent가 대화 과정에서 Neo4j를 참고하는지 테스트 합니다

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from dotenv import load_dotenv
# load the environment variables from the .env file in the root directory of the project
load_dotenv(verbose=True, override=True)
# delete the load_dotenv function from the namespace
del load_dotenv

In [78]:
graph = Neo4jGraph(
    url="bolt://localhost:7687", 
    username="neo4j", 
    password="pleaseletmein"
)

In [79]:
graph.query(
"""
MERGE (m:Movie {name:"Top Gun"})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
"""
)

[]

In [87]:
graph.refresh_schema()

In [88]:
print(graph.get_schema)


        Node properties are the following:
        []
        Relationship properties are the following:
        []
        The relationships are the following:
        []
        


In [82]:
# Query the graph
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [83]:
chain.run("Who played in Top Gun?")



> Entering new  chain...
Generated Cypher:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {name: 'Top Gun'})
RETURN a.name
Full Context:
[{'a.name': 'Tom Cruise'}, {'a.name': 'Val Kilmer'}, {'a.name': 'Anthony Edwards'}, {'a.name': 'Meg Ryan'}]

> Finished chain.


'Tom Cruise, Val Kilmer, Anthony Edwards, and Meg Ryan played in Top Gun.'

In [86]:
chain.run("Clean the graph database.")



> Entering new  chain...
Generated Cypher:
MATCH (n)
DETACH DELETE n
Full Context:
[]

> Finished chain.


"I'm sorry, but without any specific information about the graph database or the cleaning process, I'm unable to provide a helpful answer. Could you please provide more details or clarify your question?"

## Synthesize Graph Data
- Theme: Food Ingredients and Recipes

In [30]:
import json
import ast
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [89]:
synth_prompt = "Imagin a random dish. Describe the ingredients used, the cooking process, the taste of each ingredient, and the overall flavor of the dish."
synth_llm = OpenAI(temperature=0.5, verbose=True)
synth_result = synth_llm.predict(synth_prompt)
print(synth_result)



This dish is a savory and creamy mix of roasted sweet potatoes, earthy mushrooms, and nutty lentils.

The sweet potatoes are diced and roasted in olive oil and garlic until they are golden and tender. The mushrooms are sautéed in butter and seasoned with thyme, salt, and pepper. The lentils are cooked in a flavorful vegetable broth and seasoned with bay leaves and cumin.

The sweet potatoes have a caramelized, slightly sweet flavor. The mushrooms are earthy and savory. The lentils have a nutty flavor with a hint of spice from the cumin.

When all the ingredients are combined, they create a creamy, savory dish with a hint of sweetness from the sweet potatoes. The earthy mushrooms and nutty lentils combine to give the dish a rich and comforting flavor.


In [90]:
preprocess_instruction = f"주어진 텍스트에서 얻어낼 수 있는 모든 관계성 정보들을 나열하시오: {synth_result}"
preprocess_llm = OpenAI(temperature=0.5, verbose=True)
preprocess_result = preprocess_llm.predict(preprocess_instruction)
print(preprocess_result)



-Roasted sweet potatoes, earthy mushrooms, and nutty lentils are the main ingredients of the dish. 
-The sweet potatoes are diced and roasted in olive oil and garlic.
-The mushrooms are sautéed in butter and seasoned with thyme, salt, and pepper. 
-The lentils are cooked in a flavorful vegetable broth and seasoned with bay leaves and cumin.
-The sweet potatoes have a caramelized, slightly sweet flavor. 
-The mushrooms are earthy and savory. 
-The lentils have a nutty flavor with a hint of spice from the cumin.
-When all the ingredients are combined, they create a creamy, savory dish with a hint of sweetness from the sweet potatoes. 
-The earthy mushrooms and nutty lentils combine to give the dish a rich and comforting flavor.


In [91]:
knowledge_graph_parsing_instruction = f"주어진 데이터에서 얻어낼 수 있는 모든 정보들을 그래프 데이터베이스에 저장하기 위한 단위로 나누어 나열하시오: {preprocess_result}"
kg_llm = OpenAI(temperature=0.5, verbose=True)
kg_result = kg_llm.predict(knowledge_graph_parsing_instruction)
print(kg_result)



1. Ingredient: Sweet Potatoes, Mushrooms, Lentils 
2. Preparation: Dice and Roast (Sweet Potatoes), Sauté (Mushrooms), Cook (Lentils) 
3. Seasonings: Olive Oil, Garlic, Butter, Thyme, Salt, Pepper, Bay Leaves, Cumin
4. Flavor Profile: Caramelized, Sweet, Earthy, Savory, Nutty, Spicy, Creamy, Comforting 
5. Overall Flavor: Rich, Savory, Sweet


In [92]:
chain.run(f"Save the contents into the database:\n{kg_result}")



> Entering new  chain...
Generated Cypher:
CREATE (i1:Ingredient {name: "Sweet Potatoes"})
CREATE (i2:Ingredient {name: "Mushrooms"})
CREATE (i3:Ingredient {name: "Lentils"})
CREATE (p1:Preparation {name: "Dice and Roast"})
CREATE (p2:Preparation {name: "Sauté"})
CREATE (p3:Preparation {name: "Cook"})
CREATE (s1:Seasoning {name: "Olive Oil"})
CREATE (s2:Seasoning {name: "Garlic"})
CREATE (s3:Seasoning {name: "Butter"})
CREATE (s4:Seasoning {name: "Thyme"})
CREATE (s5:Seasoning {name: "Salt"})
CREATE (s6:Seasoning {name: "Pepper"})
CREATE (s7:Seasoning {name: "Bay Leaves"})
CREATE (s8:Seasoning {name: "Cumin"})
CREATE (fp1:FlavorProfile {name: "Caramelized"})
CREATE (fp2:FlavorProfile {name: "Sweet"})
CREATE (fp3:FlavorProfile {name: "Earthy"})
CREATE (fp4:FlavorProfile {name: "Savory"})
CREATE (fp5:FlavorProfile {name: "Nutty"})
CREATE (fp6:FlavorProfile {name: "Spicy"})
CREATE (fp7:FlavorProfile {name: "Creamy"})
CREATE (fp8:FlavorProfile {name: "Comforting"})
CREATE (of:OverallFlav

'To save the contents into the database, you would need to store the following information:\n\n1. Ingredient: Sweet Potatoes, Mushrooms, Lentils\n2. Preparation: Dice and Roast (Sweet Potatoes), Sauté (Mushrooms), Cook (Lentils)\n3. Seasonings: Olive Oil, Garlic, Butter, Thyme, Salt, Pepper, Bay Leaves, Cumin\n4. Flavor Profile: Caramelized, Sweet, Earthy, Savory, Nutty, Spicy, Creamy, Comforting\n5. Overall Flavor: Rich, Savory, Sweet\n\nBy saving these details, you can easily retrieve and reference them when needed.'